In [1]:
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

# Загрузка данных

In [ ]:
%%capture
!wget https://www.dropbox.com/s/64ol9q9ssggz6f1/data_ford_price.xlsx

In [55]:
data = pd.read_excel('data/data_ford_price.xlsx') 

#  Отбор признаков: мотивация

## Предобработка данных

In [60]:
data = data[['price','year', 'cylinders', 'odometer', 'lat', 'long', 'weather']]
data.dropna(inplace = True)

y = data['price']
x = data.drop(columns='price')

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=40)

## Обучение модели

In [61]:
model = LinearRegression()
model.fit(X_train, y_train)
y_predicted = model.predict(X_test)
 
mae = mean_absolute_error(y_test, y_predicted)
print('MAE: %.3f' % mae)

MAE: 4682.957


## Удаление избыточного признака

In [62]:
x.drop('lat', axis = 1, inplace = True)

In [64]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=40)

In [7]:
model = LinearRegression()
model.fit(X_train, y_train)
y_predicted = model.predict(X_test)
 
mae = mean_absolute_error(y_test, y_predicted)
print('MAE: %.3f' % mae)

MAE: 4672.930


#  Отбор признаков: классификация методов

## Метод рекурсивного исключения признаков

In [65]:
from sklearn.feature_selection import RFE

In [66]:
y = data['price']
x = data.drop(columns='price')

In [67]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=40)

In [68]:
estimator = LinearRegression()
selector = RFE(estimator, n_features_to_select=3, step=1)
selector = selector.fit(X_train, y_train)
 
selector.get_feature_names_out()

array(['year', 'cylinders', 'lat'], dtype=object)

In [12]:
X_train.columns

Index(['year', 'cylinders', 'odometer', 'lat', 'long', 'weather'], dtype='object')

In [13]:
selector.ranking_

array([1, 1, 4, 1, 3, 2])

##  МЕТОДЫ ВЫБОРА ПРИЗНАКОВ НА ОСНОВЕ ФИЛЬТРОВ

In [14]:
from sklearn.feature_selection import SelectKBest, f_regression

In [16]:
selector = SelectKBest(f_regression, k=3)
selector.fit(X_train, y_train)
 
selector.get_feature_names_out()

array(['year', 'cylinders', 'odometer'], dtype=object)

**Задание 9.4**

In [17]:
from sklearn.feature_selection import SelectKBest, f_classif

selector = SelectKBest(f_classif, k=3)
selector.fit(X_train, y_train)
 
selector.get_feature_names_out()

array(['year', 'cylinders', 'odometer'], dtype=object)

In [19]:
data.head(1)

,price,year,cylinders,odometer,lat,long,weather
0,43900,2016,6,43500,36.4715,-82.4834,59.0


In [26]:
url = 'https://github.com/Nitys70/LESSONS_FILES/raw/refs/heads/main/ML-6/data/data_ford_price.xlsx'

data_ford = pd.read_excel(url)

data_ford.head(1)

,price,year,condition,cylinders,odometer,title_status,transmission,drive,size,lat,long,weather
0,43900,2016,4,6,43500,clean,automatic,4wd,full-size,36.4715,-82.4834,59.0


In [98]:
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

from sklearn.feature_selection import RFE
from sklearn.feature_selection import SelectKBest, f_regression, f_classif

# Чтение файла
# url = 'https://github.com/Nitys70/LESSONS_FILES/raw/refs/heads/main/ML-6/data/data_ford_price.xlsx'

# data_ford = pd.read_excel(url)

data_ford = pd.read_excel('data/data_ford_price.xlsx')



# Предварительный отбор признаков
data_ford = data_ford[['price','year', 'cylinders', 'odometer', 'lat', 'long', 'weather']]
data_ford.dropna(inplace = True)

# Делим датасет на таргетное значение и предикторы, 
# формируем тренировочную и тестовую выборки
y = data_ford['price']
x = data_ford.drop(columns='price')

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)


# Прибегаем к методу рекурсивного исключения признаков (RFE), формируем список признаков
estimator = LinearRegression()
selector_rfe = RFE(estimator, n_features_to_select=3, step=1)
selector_rfe = selector_rfe.fit(X_train, y_train)
 
rfe_list = list(selector_rfe.get_feature_names_out())


# Прибегаем к методу отбора признаков с помощью коэф-та корелляции Спирмена, 
# формируем список признаков
selector_f_reg = SelectKBest(f_regression, k=3)
selector_f_reg.fit(X_train, y_train)
 
f_reg_list = list(selector_f_reg.get_feature_names_out())

# Прибегаем к методу отбора признаков с помощью коэф-та корелляции Спирмена, 
# формируем список признаков
selector_f_classif = SelectKBest(f_classif, k=3)
selector_f_classif.fit(X_train, y_train)
 
f_classif_list = list(selector_f_classif.get_feature_names_out())

# На основе полученных списков формируем словарь (по нему далее пойдем циклом)
feats_top_dict = {'RFE Selection': rfe_list,
                  'KBest selection f_regression': f_reg_list,
                  'KBest selection f_classif': f_classif_list
                  }

# Задаем пустой словарь - сюда занесем результаты метрик MAE различных моделей
mae_results = dict()

# Циклом идем по созданным элементам словаря и обучаем модель, считаем метрики
for key, value in feats_top_dict.items():
    
    # Формируем DF предикторов (исходя из результатов отбора признаков), таргета
    x = data_ford[value]
    y = data_ford['price']
    
    # Делим на тренировочную и тестовую выборки
    X_train, X_test, y_train, y_test = train_test_split(x, y, train_size=0.3, random_state=42)
    
    # Создаем модель, обучаем ее, делаем прогноз, считаем метрику
    model = LinearRegression()
    
    model.fit(X_train, y_train)
    
    y_predicted = model.predict(X_test)
    
    mae = round(mean_absolute_error(y_test, y_predicted), 3)
    
    # Заносим в словарь название метода отбора признака и MAE, полученного на этих признаках
    mae_results[key] = mae
    
    
# Формируем из полученного словаря с метриками MAE DF, фильтруем по убыванию
mae_results_df = pd.DataFrame.from_dict(mae_results, orient='index')
mae_results_df = mae_results_df.sort_values(0)
mae_results_df = mae_results_df.reset_index()
mae_results_df.columns = ['Selection method', 'MAE Value']

# Выводим на печать
display(mae_results_df)


,Selection method,MAE Value
0,KBest selection f_regression,4606.024
1,KBest selection f_classif,4606.024
2,RFE Selection,5213.692


Мы видим два одинаковых значения метрик. Вероятно, методами KBest были отобраны одни и те же признаки. Проверим это. 

In [97]:
f_reg_list == f_classif_list

True

Действительно, одинаковые метрики получены в результате идентичности отобранных придикторов.

**Вывод:**

Наилучшая метрика средней абсолютной ошибки **(MAE: 4606.024)** получена у модели, обученной на предикторах, которые мы получили при применении метода выбора признаков на основе статистических мер. Причем, при применении коэффциента корреляции Пирсона и дисперсного анализа получены идентичные значения. 

Так же, важно отметить, что прирост метрики наблюдается и относительно моделей, обученных на всех числовых признаках, лишенных мультиколлинеарности **(MAE: 4672.930)**.

Обратный эффект (ухудщение метрики) при использовании отбора предикторов методом рекурсивного исключения признаков **(MAE: 5213.692)**. Здесь значение метрики существенно снизилось. 

In [70]:
rfe_list

['year', 'cylinders', 'lat']

In [78]:
mae_results
# mae_results_list

{'RFE Selection': 5213.692259173624,
 'KBest selection f_regression': 4606.024237484877,
 'KBest selection f_classif': 4606.024237484877}

In [42]:
test_dict = {5:'fasag', 1: 'df', 2: 'fads', 0:'fasfas'}

test_dict[list(sorted(test_dict.keys()))[-1]]

'fasag'